In [1]:
# The Main Function for APAD Project -- Cloth On Fly

# Author: Tianyi (Kelly) Zhang
#         Shikha Singh

# Creation Date: 07/21/2019

# Major: MSITM

In [19]:
import sqlite3
from datetime import date
from datetime import timedelta,date, datetime
import csv
import glob

In [20]:
try:
    db = sqlite3.connect("ClothONFly.db")
    cursor = db.cursor()
except Error as e:
    print(e)

In [21]:
#     def create_tables(self):
#         connect = Database.initialize()
#         db = connect.cursor()
#         db.execute(
#             '''CREATE TABLE Users(User_ID INTEGER PRIMARY KEY AutoIncrement,
#                                 User_Name Text NOT NULL UNIQUE,
#                                 Password Text,
#                                 User_Role Text Not Null check(User_Type = "Admin" or User_Type = "Buyer" or User_Type = "Seller"),
#                                 First_Name TEXT NOT NULL,
#                                 Last_Name TEXT NOT NULL, 
#                                 Email TEXT Not NUll, 
#                                 Address Text Not NUll, 
#                                 Phone_Num INTEGER Not Null) ''')

#         db.execute('''
#             CREATE TABLE Inventory_Items(Item_ID INTEGER PRIMARY KEY AutoIncrement,
#                                         Brand_Name TEXT NOT NULL,
#                                         Type TEXT NOT NULL,
#                                         Size TEXT NOT NULL,
#                                         Gender TEXT NOT NULL check(Gender = "Male" or Gender = "Female"),
#                                         Original_Price REAL NOT NULL,
#                                         Rental_Price REAL NOT NULL,
#                                         Available_From DATE,
#                                         Owner_ID INTEGER,
#                                         Location TEXT NOT NULL,
#                                         Cloth_Image BLOB NOT NULL,
#                                         Deposit REAL NOT NULL,
#                                         Foreign Key(Owner_ID) References Users(User_ID))
#         ''')

#         db.execute(
#             '''CREATE TABLE Orders (Order_ID INTEGER PRIMARY KEY AutoIncrement, 
#                                     User_ID INTEGER NOT NULL, 
#                                     Item_ID INTEGER Not Null,
#                                     Delivery_Date Date Not Null, 
#                                     Return_Date Date Not Null, 
#                                     Shipping_Address Text Not Null,
#                                     Order_Status Text Not Null check(Order_Status = "Order Received(Shipment Pending)" or Order_Status = "Order Shipped" or Order_Status = "Return Initiated" or Order_Status = "Return Received(Deposit Refunded)" or Order_Status = "Return Not Received(Deposit Withheld)"),
#                                     Shipping_Date Date, 
#                                     FOREIGN KEY(User_ID) REFERENCES Users(User_ID), 
#                                     FOREIGN KEY(Item_ID) References Inventory_Items(Item_ID))
#             ''')

In [22]:
db.commit()
cursor.close()
db.close()

activities.py

In [23]:
def register(user_name, password, user_role, first_name, last_name, email, address, phone_num, id_num) -> bool:

    if user_role.lower() == ADMIN_ROLE_NAME.lower():
        new_admin_user = Admin(user_name, password, first_name,
                               last_name, email, address, phone_num, id_num)
        write_user_into_database(ADMIN_ROLE_NAME, new_admin_user)
        print("Register succeeded! Returning to the main menu...\n")
        
    elif user_role.lower() == SELLER_ROLE_NAME.lower():
        new_seller_user = Seller(user_name, password, first_name,
                               last_name, email, address, phone_num, id_num)
        move_seller_to_pending(new_seller_user)
        print("Your registration is pending for authorization, thank you for your patience.")
        
    elif user_role.lower() == BUYER_ROLE_NAME.lower():
        print('This user is a buyer.')
        new_buyer_user = Buyer(user_name, password, first_name,
                                 last_name, email, address, phone_num, id_num)
        print('Writing the new buyer into db...')
        write_user_into_database(BUYER_ROLE_NAME, new_buyer_user)
        print("Register succeeded! Returning to the main menu...\n")
        
    else:
        print("Unknown role. Please try again.\n")
        return False

    return True

In [24]:
def check_duplicate(user_name):
    db = sqlite3.connect("ClothONFly.db")
    cursor = db.cursor()
    try:
        cur_result = cursor.execute('''SELECT * FROM Users where User_Name = ? ''',(str(user_name),))
        db.commit()
        if len(cur_result.fetchall()) != 0:
            cursor.close()
            db.close()
            return True
        else:
            cursor.close()
            db.close()
            return False
    except Exception as e:
        cursor.close()
        db.close()
        print(e)
        return True

In [25]:
def login(user_name, password):
    user_name = TEST_ADMIN_NAME
    password = TEST_PASSWORD
    try: 
        db = sqlite3.connect("ClothONFly.db")
        cursor = db.cursor()
        cur_result = cursor.execute('''SELECT * FROM Users where User_Name = ? AND Password = ? ''',(user_name, password))
        db.commit()
        if len(cur_result.fetchall()) != 0:
            print("LogIn Succeed")
            cursor.close()
            db.close()
            return True
        else:
            print("Unable to LogIn")
            cursor.close()
            db.close()
            return False
        print("db is closed!")
        return True
    except Exception as e:
        cursor.close()
        db.close()
        print(e)
        return False

In [26]:
def return_role(user_name):
    db = sqlite3.connect("ClothONFly.db")
    cursor = db.cursor()
    
    try:
        cur_result = cursor.execute('''SELECT User_Type FROM Users where User_Name = ? ''', (str(user_name),))
        cur_result_all = cur_result.fetchall()
        db.commit()
        cursor.close()
        db.close()
        
        for row in cur_result_all:
            return row[0]
        
    except Exception as e:
        cursor.close()
        db.close()
        print(e)

In [27]:
def write_user_into_database(user_role, user) -> bool:
    try: 
        db = sqlite3.connect("ClothONFly.db")
        cursor = db.cursor()
        cursor.execute('''Insert into Users(User_Name, Password, User_Type, First_Name, Last_Name, Email, Address, Phone_Num, User_ID) VALUES(?,?,?,?,?,?,?,?,?)''',
                       (user.user_name, user.password, user_role, user.first_name, user.last_name, user.email, user.address, user.phone_num, None))
        db.commit()
        cursor.close()
        db.close()
        print("db is closed!")
        return True
    except Exception as e:
        cursor.close()
        db.close()
        print(e)
        return False

In [28]:
PENDING_SELLER_LIST = list()
def move_seller_to_pending(seller) -> bool:
    PENDING_SELLER_LIST.append(seller)
    return True

In [29]:
def get_userid_from_username(user_name):
    print("get_userid_by_username")
    try: 
        db = sqlite3.connect("ClothONFly.db")
        cursor = db.cursor()
        cursor.execute('''SELECT User_ID FROM Users WHERE User_Name = ?''', (user_name,))
        db.commit()
        cursor.close()
        db.close()
        print("db is closed!")
        return True
    except Exception as e:
        cursor.close()
        db.close()
        print(e)
        return False

In [30]:
class Admin:

    def __init__(self, user_name, password, first_name, last_name, email, address, phone_num, id_num):
        self.user_name = user_name
        self.password = password
        self.first_name = first_name
        self.last_name = last_name
        self.email = email
        self.address = address
        self.phone_num = phone_num
        self.id_num = id_num
    
    @staticmethod
    def authorize_sellers() -> bool:
        print("Hi Admin! Please check and authorize the following sellers for registration:")
        
        while (len(PENDING_SELLER_LIST) > 0):
            seller = PENDING_SELLER_LIST[0]
            print('PENDING_SELLER_LIST size: ' + str(len(PENDING_SELLER_LIST)) + "\n")
            # Notify the current admin about the seller's info
            print("Seller basic: \n", 
                  ((seller.user_name), (seller.password), (SELLER_ROLE_NAME), 
                   (seller.first_name), (seller.last_name), (seller.email), 
                   (seller.address), (seller.phone_num), (seller.id_num)))
            decision = input('Do you want to grant this seller permission to the system (Y/N)? ')
            if decision.lower() == 'y':
                print("Writing seller into database...")
                write_user_into_database(SELLER_ROLE_NAME, seller)
                print("Finished.\n")

            print("Removing the seller from the pending list...")
            PENDING_SELLER_LIST.remove(seller)
            print('PENDING_SELLER_LIST size: ' + str(len(PENDING_SELLER_LIST)) + "\n")
            
        print("Back to Admin Menu...\n")
        return True

    @staticmethod
    # Remove a user from the user table if needed
    def remove_user() -> bool:
        answer = input("Please input the user name you want to remove: ")
        db = sqlite3.connect("ClothONFly.db")
        cursor = db.cursor()
        try:
            cursor.execute('''DELETE FROM Users 
                              WHERE User_Name = ?''',(answer,))
            db.commit()
            cursor.close()
            db.close()
            return True
        except Exception as e:
            cursor.close()
            db.close()
            print(e)
            return False

        return False

seller.py

In [ ]:

    image_dir = "Images/"
    export_dir = "Exports/"
    import_dir = "ImportFiles/"

    @staticmethod
    def add_item(Brand_Name,Type,Size,Gender,Original_Price,Rental_Price,Owner_ID,Location,Cloth_Image,Deposit,Available_From=date.today()) :
# 
#         Description goes here

# 
        db = Database.Database.initialize()
        cursor = db.cursor()

        for image in glob.glob(Seller.image_dir+'/*'):
            if (str(image).replace(Seller.image_dir,'')) == Cloth_Image:
                print(str(image))
                with open(image, 'rb') as file:
                    blobData = file.read()

        cursor.execute(
            '''Insert into Inventory_Items(Brand_Name,Type,Size,Gender,Original_Price,Rental_Price,Owner_ID,Location,Cloth_Image,Deposit,Available_From) values(?,?,?,?,?,?,?,?,?,?,?)''',
            (Brand_Name,Type,Size,Gender,Original_Price,Rental_Price,Owner_ID,Location,blobData,Deposit,Available_From))
        db.commit()

        Database.Database.close_connection()

    @staticmethod
    def remove_item(Item_ID):
#         '''
#         Description goes here
#         '''
        db = Database.Database.initialize()
        cursor = db.cursor()
        cursor.execute('''Delete from Inventory_Items where Item_ID == ?''',(str(Item_ID)))
        db.commit()
        Database.Database.close_connection()

    @staticmethod
    def update_item(Item_ID,Brand_Name,Type,Size,Gender,Original_Price,Rental_Price,Owner_ID,Location,Cloth_Image,Deposit,Available_From):
#         '''
#         Description goes here
#         '''
        db = Database.Database.initialize()
        cursor = db.cursor()

        for image in glob.glob(Seller.image_dir + '/*'):
            if (str(image).replace(Seller.image_dir, '')) == Cloth_Image:
                with open(image, 'rb') as file:
                    blobData = file.read()
        cursor.execute('''Update Inventory_Items 
        set Brand_Name= ?,
        Type = ? ,
        Size =?,
        Gender = ?,
        Original_Price =?,
        Rental_Price =?,
        Owner_ID=?,
        Location=?,
        Cloth_Image=?,
        Deposit=?,
        Available_From=? where Item_ID = ?''',(Brand_Name,Type,Size,Gender,Original_Price,Rental_Price,Owner_ID,Location,blobData,Deposit,Available_From,Item_ID))
        db.commit()
        Database.Database.close_connection()

    @staticmethod
    def ship_item(Order_ID):
        db = Database.Database.initialize()
        cursor = db.cursor()
        cursor.execute('''Update Orders 
                set Order_Status = 'Order Shipped',
                    Shipping_Date = ?
                where Order_ID = ?''', (date.today(),Order_ID))
        db.commit()
        Database.Database.close_connection()

    @staticmethod
    def refund_deposit(Order_ID):
        db = Database.Database.initialize()
        cursor = db.cursor()
        cursor.execute('''Update Orders 
                    set Order_Status = 'Return Received(Deposit Refunded)'
                    where Order_ID = ? and Return_Date >= ? and Order_Status = "Return Initiated" ''', (str(Order_ID),date.today()))
        db.commit()
        Database.Database.close_connection()

    @staticmethod
    def withhold_deposit(Order_ID):
        db = Database.Database.initialize()
        cursor = db.cursor()
        cursor.execute('''Update Orders 
                        set Order_Status = 'Return Not Received(Deposit Withheld)'
                        where Order_ID = ? and Return_Date < ? and Order_Status != "Return Initiated" ''', (str(Order_ID),date.today()))
        db.commit()
        Database.Database.close_connection()

    @staticmethod
    def process_bulk_refunds():
        return None

    @staticmethod
    def bulk_item_upload(filename):
        for files in glob.glob(Seller.import_dir + '/*'):
            if (str(files).replace(Seller.import_dir, '')) == filename:
                with open(Seller.import_dir + '/'+filename, 'r') as inputfile:
                    items = csv.DictReader(inputfile)

                    for item in items:
                        if item['AvailableFrom'] == '':
                            item['AvailableFrom'] = date.today()
                        Seller().add_item(item['Brand'],item['Type'],item['Size'],item['Gender'],item['OriginalPrice'],item['RentalPrice'],item['OwnerID'],item['Location'],item['ClothImage'],item['Deposit'],item['AvailableFrom'])

    @staticmethod
    def all_orders():

        db = Database.Database.initialize()
        cursor = db.cursor()
        cursor.execute('''select o.Order_ID,o.Order_Status As "Order Status", o.Delivery_Date As "Delivery Date",o.Return_Date As "Return Date",o.Shipping_Address AS "Shipping Address",
                                u.First_Name AS "Buyer's First Name",u.Last_Name AS "Buyer's LastName",u.Email AS "Buyer's Email",u.Phone_Num AS "Buyer's Contact",
                                i.Brand_Name AS "Product's Brand",i.Type AS "Product Type", i.Size AS "Product Size",i.Gender AS "Gender",
                                usr.First_Name as "Seller's first Name", usr.Last_Name as "Seller's Last Name", usr.Email As "Seller's Email",usr.Phone_Num AS "Seller's Contact"
                                from Orders o join Users u on u.User_ID=o.User_ID join Inventory_Items i on o.Item_ID = i.Item_ID join Users usr on i.Owner_ID =usr.User_ID ''')

        with open(Seller.export_dir+'/'+"Orders_export_"+ str(date.today()) + ".csv", "w", newline='') as file:
            csv_writer = csv.writer(file)
            csv_writer.writerow([i[0] for i in cursor.description])  # write headers
            csv_writer.writerows(cursor)
            
    @staticmethod
    def orders_per_seller(Seller_ID):

        db = Database.Database.initialize()
        cursor = db.cursor()
        cursor.execute('''select User_Name from Users where User_ID = ?''',(Seller_ID,))
        for row in cursor.fetchall():
            seller = row[0]
        cursor.execute('''select o.Order_ID,o.Order_Status As "Order Status", o.Delivery_Date As "Delivery Date",o.Return_Date As "Return Date",o.Shipping_Address AS "Shipping Address",
                                        u.First_Name AS "Buyer's First Name",u.Last_Name AS "Buyer's LastName",u.Email AS "Buyer's Email",u.Phone_Num AS "Buyer's Contact",
                                        i.Brand_Name AS "Product's Brand",i.Type AS "Product Type", i.Size AS "Product Size",i.Gender AS "Gender",
                                        usr.First_Name as "Seller's first Name", usr.Last_Name as "Seller's Last Name", usr.Email As "Seller's Email",usr.Phone_Num AS "Seller's Contact"
                                        from Orders o join Users u on u.User_ID=o.User_ID join Inventory_Items i on o.Item_ID = i.Item_ID join Users usr on i.Owner_ID =usr.User_ID 
                                        where usr.User_ID = ? ''', (Seller_ID,))



        with open(Seller.export_dir + '/' + seller + "'s_Orders_export_" + str(date.today()) + ".csv", "w", newline='') as file:
            csv_writer = csv.writer(file)
            csv_writer.writerow([i[0] for i in cursor.description])  # write headers
            csv_writer.writerows(cursor)